<a href="https://colab.research.google.com/github/awesome-astra/docs/blob/main/docs/pages/tools/notebooks/Retrieval_Augmented_Generation_(for_AI_Chatbots).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Astra Hybrid Search Example**

**Goal:** Comparison of a Q&A example against ChatGPT utilizing retrieval augmentation (RAG) to retrieve relevant information from an external knowledge base and give that information to the LLM. In doing so, compare hybrid search - using term-based and vector-based search - with vector only search.

# **Install libraries and import modules**

In [ ]:
!pip install openai pandas jupyter-datatables cassandra-driver datasets

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
import numpy
import pandas as pd
import time
from getpass import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

# **Keys & Environment Variables**

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass('Your Astra DB Token ("AstraCS:..."): ')

Your Astra DB Token ("AstraCS:..."): ··········


In [ ]:
# Input your OpenAI api key
openai_api_key = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [ ]:
# Input your database keyspace name:
my_ks = input('Your Astra Keyspace name: ')

Your Astra Keyspace name: vector_preview


In [ ]:
# keys and tokens here
openai.api_key = openai_api_key
cass_user = 'token'
cass_pw = ASTRA_DB_TOKEN_BASED_PASSWORD

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    scb_path = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-vectordemo.zip to secure-connect-vectordemo.zip


# **Select a model to compute embeddings**

In [ ]:
model_id = "text-embedding-ada-002"

# **Connect to Astra with Hybrid Search**

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace(my_ks)
session

# **Drop / Create Schema**

In [ ]:
# only use this to reset the schema
session.execute(f"""DROP INDEX IF EXISTS title_context_vector""")
session.execute(f"""DROP INDEX IF EXISTS context_term""")
session.execute(f"""DROP TABLE IF EXISTS squad_hybrid""")

In [ ]:
# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS squad_hybrid
(id text,
 title text,
 context text,
 question text,
 answers map<text,text>,
 title_context_embedding vector<float, 1536>,
 PRIMARY KEY (id,title))""")

# # Create Indexes

# First - Vector index
session.execute("""
    CREATE CUSTOM INDEX IF NOT EXISTS title_context_vector
    ON squad_hybrid (title_context_embedding)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {'similarity_function' : 'dot_product'}
""")

# Second - Text Analyzer index
session.execute("""
    CREATE CUSTOM INDEX IF NOT EXISTS context_term
    ON squad_hybrid (context)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {
      'index_analyzer': '{
      "tokenizer" : {"name" : "standard"},
      "filters" : [{"name" : "porterstem"}]
      }'}
""")


# **Loading the Data**

Start by constructing our knowledge base. Download a mostly prepared dataset called Stanford Question-Answering Dataset (SQuAD) hosted on Hugging Face Datasets.

In [ ]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

Let's see the 1st 5 records. The dataset does contain duplicate contexts,

In [ ]:
data = data.to_pandas()
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


Remove the duplicate context and keep the 1st record of the records with same context.

In [ ]:
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."


# **Load the table with data and create text embeddings**

* If you are using free trial of the OpenAPI,  the rate limit is about 60/min.  Please adjust the ***counter*** variable below according to your rate limit.
* It takes a long time to load the full data so it is set as 300 here.   Please adjust the ***total*** variable below according to the number of data to test.  Please note that the more data is loaded the longer it takes to load.

In [ ]:
counter = 0;
total = 0
for id, row in data.iterrows():

  # This is to convert the content of 'answers' to be map<text,text> so it can be saved into Astra.
  converted_answers = dict()
  converted_answers['text'] = row.answers['text'][0]
  converted_answers['answer_start'] = str(row.answers['answer_start'][0])

  # Create Embedding for each conversation row, save them to the database
  full_chunk = f"{row.context} {row.title}"
  embedding = openai.Embedding.create(input=full_chunk, model=model_id)['data'][0]['embedding']

  query_hybrid = SimpleStatement(
                f"""
                INSERT INTO squad_hybrid
                (id, title, context, question, answers, title_context_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )

  display(row)

  session.execute(query_hybrid, (row.id, row.title, row.context, row.question, converted_answers, embedding))

  # With free trial of openAI, the rate limit is set as 60/per min.  Please set this counter depends on your own rate limit.
  counter += 1

  # It takes a long time to load all data.  It is set as 300 so it takes a few mins to load.
  total += 1

  if(total >= 300):
    break

  # With OpenAPI free trial,  the rate limit is 60 / per min.   So when hit 60 requests,  the program will sleep for 60 seconds.
  if (counter >= 300):
    counter = 0;
    time.sleep(60)


id                                   5733be284776f41900661182
title                                University_of_Notre_Dame
context     Architecturally, the school has a Catholic cha...
question    To whom did the Virgin Mary allegedly appear i...
answers     {'text': ['Saint Bernadette Soubirous'], 'answ...
Name: 0, dtype: object

id                                   5733bf84d058e614000b61be
title                                University_of_Notre_Dame
context     As at most other universities, Notre Dame's st...
question    When did the Scholastic Magazine of Notre dame...
answers     {'text': ['September 1876'], 'answer_start': [...
Name: 5, dtype: object

id                                   5733bed24776f41900661188
title                                University_of_Notre_Dame
context     The university is the major seat of the Congre...
question    Where is the headquarters of the Congregation ...
answers             {'text': ['Rome'], 'answer_start': [119]}
Name: 10, dtype: object

id                                   5733a6424776f41900660f51
title                                University_of_Notre_Dame
context     The College of Engineering was established in ...
question    How many BS level degrees are offered in the C...
answers            {'text': ['eight'], 'answer_start': [487]}
Name: 15, dtype: object

id                                   5733a70c4776f41900660f64
title                                University_of_Notre_Dame
context     All of Notre Dame's undergraduate students are...
question    What entity provides help with the management ...
answers     {'text': ['Learning Resource Center'], 'answer...
Name: 20, dtype: object

...

id                                   56cffba5234ae51400d9c1f1
title                                         Frédéric_Chopin
context     A reconciliation of these views is suggested b...
question    William Atwood suggested that Chopin's music w...
answers        {'text': ['intuitive'], 'answer_start': [443]}
Name: 2161, dtype: object

id                                   56cffc2b234ae51400d9c1ff
title                                         Frédéric_Chopin
context     Jones comments that "Chopin's unique position ...
question    Arthur Hutchings stated that Chopin's lack of ...
answers     {'text': ['Byronic flamboyance'], 'answer_star...
Name: 2164, dtype: object

id                                   56cffcf3234ae51400d9c20d
title                                         Frédéric_Chopin
context     Chopin's qualities as a pianist and composer w...
question    In what suite did Schumann name a work for Cho...
answers         {'text': ['Carnaval'], 'answer_start': [138]}
Name: 2168, dtype: object

id                                   56cffdaa234ae51400d9c226
title                                         Frédéric_Chopin
context     Two of Chopin's long-standing pupils, Karol Mi...
question       Who dedicated his 1915 piano Études to Chopin?
answers          {'text': ['Debussy'], 'answer_start': [392]}
Name: 2176, dtype: object

id                                   56cc239e6d243a140015eeb7
title          Sino-Tibetan_relations_during_the_Ming_dynasty
context     The exact nature of relations between Tibet an...
question            Who were Wang Jiawei and Nyima Gyaincain?
answers     {'text': ['Mainland Chinese scholars'], 'answe...
Name: 2180, dtype: object

# **Convert a query string into a text embedding to use as part of the query**

Provide a question to see how it will work with hybrid search and with vector search alone.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [ ]:
# Question to find out the information that you need.
customer_input = "What degrees are offered?"
term_search = "Notre Dame"

# Create embedding based on same model
embedding = openai.Embedding.create(input=customer_input, model=model_id)['data'][0]['embedding']
display(embedding)

[0.011705282144248486,
 -0.010144149884581566,
 -0.003181691048666835,
 -0.03600241616368294,
 -0.04453403875231743,
 0.028164630755782127,
 -0.026571376249194145,
 0.024515563622117043,
 0.0006219635833986104,
 0.0018534434493631124,
 0.002682192949578166,
 0.010728771798312664,
 -0.015996791422367096,
 0.004342903848737478,
 0.014801849611103535,
 -0.006758483592420816,
 0.025286493822932243,
 0.0010793015826493502,
 0.0019353547831997275,
 -0.020738009363412857,
 -0.010799439623951912,
 0.000582212524022907,
 -0.01536719873547554,
 -0.002799438312649727,
 -0.01532865222543478,
 0.02240835689008236,
 0.009912870824337006,
 0.0001348123187199235,
 -0.0019385669147595763,
 -0.014326442964375019,
 0.034409161657094955,
 0.012257781811058521,
 -0.04527927190065384,
 -0.021534636616706848,
 -0.008795022964477539,
 -0.004468179773539305,
 -0.022421205416321754,
 0.004024895373731852,
 0.006222045049071312,
 0.028370212763547897,
 0.007889180444180965,
 0.01548283826559782,
 -0.001158000668

**Hybrid query**
Let's take a look at what a hybrid query against they database would look like. The query vector has the same dimensions (number of entries in the list) as the embeddings we generated for each row in the database. The term search, specifying the university name has also been added to better refine the search.

In [ ]:
# Build the hybrid search query. For vector search cosine similarity distance calculation, use the embedding to find the information nearest to the question asked.
search_hybrid = SimpleStatement(
    f"""
    SELECT id, title, answers, context, question
    FROM squad_hybrid
    WHERE context : '{term_search}'
    ORDER BY title_context_embedding ANN OF {embedding} LIMIT 5;
    """
    )
display(search_hybrid)

<SimpleStatement query="
    SELECT id, title, answers, context, question
    FROM squad_hybrid
    WHERE context : 'Notre Dame'
    ORDER BY title_context_embedding ANN OF [0.011705282144248486, -0.010144149884581566, -0.003181691048666835, -0.03600241616368294, -0.04453403875231743, 0.028164630755782127, -0.026571376249194145, 0.024515563622117043, 0.0006219635833986104, 0.0018534434493631124, 0.002682192949578166, 0.010728771798312664, -0.015996791422367096, 0.004342903848737478, 0.014801849611103535, -0.006758483592420816, 0.025286493822932243, 0.0010793015826493502, 0.0019353547831997275, -0.020738009363412857, -0.010799439623951912, 0.000582212524022907, -0.01536719873547554, -0.002799438312649727, -0.01532865222543478, 0.02240835689008236, 0.009912870824337006, 0.0001348123187199235, -0.0019385669147595763, -0.014326442964375019, 0.034409161657094955, 0.012257781811058521, -0.04527927190065384, -0.021534636616706848, -0.008795022964477539, -0.004468179773539305, -0.0224212054163

And provide a vector only query to compare that hybrid search with a vector only search.

In [ ]:
# Compare that with only vector search,using only the embedding to find the information nearest to the question asked.
search_vector = SimpleStatement(
    f"""
    SELECT id, title, answers, context, question
    FROM squad_hybrid
    ORDER BY title_context_embedding ANN OF {embedding} LIMIT 5;
    """
    )
display(search_vector)

<SimpleStatement query="
    SELECT id, title, answers, context, question
    FROM squad_hybrid
    ORDER BY title_context_embedding ANN OF [0.011705282144248486, -0.010144149884581566, -0.003181691048666835, -0.03600241616368294, -0.04453403875231743, 0.028164630755782127, -0.026571376249194145, 0.024515563622117043, 0.0006219635833986104, 0.0018534434493631124, 0.002682192949578166, 0.010728771798312664, -0.015996791422367096, 0.004342903848737478, 0.014801849611103535, -0.006758483592420816, 0.025286493822932243, 0.0010793015826493502, 0.0019353547831997275, -0.020738009363412857, -0.010799439623951912, 0.000582212524022907, -0.01536719873547554, -0.002799438312649727, -0.01532865222543478, 0.02240835689008236, 0.009912870824337006, 0.0001348123187199235, -0.0019385669147595763, -0.014326442964375019, 0.034409161657094955, 0.012257781811058521, -0.04527927190065384, -0.021534636616706848, -0.008795022964477539, -0.004468179773539305, -0.022421205416321754, 0.004024895373731852, 0.00

# **Find the top results using hybrid (Term and ANN similarity search)**

In [ ]:
results = session.execute(search_hybrid)
top_hybrid_results = results._current_rows

for row in top_hybrid_results:
  print(f"""{row.context}\n""")

The School of Architecture was established in 1899, although degrees in architecture were first awarded by the university in 1898. Today the school, housed in Bond Hall, offers a five-year undergraduate program leading to the Bachelor of Architecture degree. All undergraduate students study the third year of the program in Rome. The university is globally recognized for its Notre Dame School of Architecture, a faculty that teaches (pre-modernist) traditional and classical architecture and urban planning (e.g. following the principles of New Urbanism and New Classical Architecture). It also awards the renowned annual Driehaus Architecture Prize.

Besides its prominence in sports, Notre Dame is also a large, four-year, highly residential research University, and is consistently ranked among the top twenty universities in the United States  and as a major global university. The undergraduate component of the university is organized into four colleges (Arts and Letters, Science, Engineerin

# **Find the top results only using ANN Similarity (Vector)**

In [ ]:
results = session.execute(search_vector)
top_vector_results = results._current_rows

for row in top_vector_results:
  print(f"""{row.context}\n""")

The university first offered graduate degrees, in the form of a Master of Arts (MA), in the 1854–1855 academic year. The program expanded to include Master of Laws (LL.M.) and Master of Civil Engineering in its early stages of growth, before a formal graduate school education was developed with a thesis not required to receive the degrees. This changed in 1924 with formal requirements developed for graduate degrees, including offering Doctorate (PhD) degrees. Today each of the five colleges offer graduate education. Most of the departments from the College of Arts and Letters offer PhD programs, while a professional Master of Divinity (M.Div.) program also exists. All of the departments in the College of Science offer PhD programs, except for the Department of Pre-Professional Studies. The School of Architecture offers a Master of Architecture, while each of the departments of the College of Engineering offer PhD programs. The College of Business offers multiple professional programs i

# **Ask ChatGPT for some help**

* Here we build a prompt with which we'll query ChatGPT.
* We'll pass once with the hybrid search output and once with the vector only search output - to compare the results
* Note the "roles" in this little conversation give the LLM more context about who that part of the conversation is coming from.
* This may take 10-20 seconds to return, so be patient.

In [ ]:
# Hybrid results

message_objects = []

# With the role as 'system',  we tell the model how we want it to behave and tell it how its personality and type of response should be.
message_objects.append({"role":"system",
                        "content":"You're a chatbot helping customers with questions."})

# With the role as 'user',  pass the question from user.
message_objects.append({"role":"user",
                        "content": customer_input})

answers_list = []

# With the role as 'assistant',  load the results from Astra with Vector Search.  That helps the model to provide answer to the question asked by user.
for row in top_hybrid_results:
    brand_dict = {'role': "assistant", "content": f"{row.context}"}
    answers_list.append(brand_dict)

message_objects.extend(answers_list)
message_objects.append({"role": "assistant", "content":"Here's my answer to your question."})

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)

print(completion.choices[0].message['content'])

The University of Notre Dame offers a wide range of undergraduate and graduate degrees in various fields. The undergraduate programs include degrees in Arts and Letters, Business, Science, and Engineering. Within these colleges, you can choose from a variety of majors and minors to pursue your specific interests. 

At the graduate level, Notre Dame offers a variety of master's and doctoral programs across different disciplines, such as humanities, social sciences, natural sciences, engineering, business, architecture, and law. These programs provide advanced education and research opportunities for further specialization in your field of study.

Additionally, Notre Dame offers professional degree programs in areas like law and business administration. These programs are designed to provide specialized training and skills necessary for specific careers.

Overall, Notre Dame has a diverse range of degree programs to cater to the interests and career goals of its students.


In [ ]:
# Vector search only results

message_objects = []

# With the role as 'system',  we tell the model how we want it to behave and tell it how its personality and type of response should be.
message_objects.append({"role":"system",
                        "content":"You're a chatbot helping customers with questions."})

# With the role as 'user',  pass the question from user.
message_objects.append({"role":"user",
                        "content": customer_input})

answers_list = []

# With the role as 'assistant',  load the results from Astra with Vector Search.  That helps the model to provide answer to the question asked by user.
for row in top_vector_results:
    brand_dict = {'role': "assistant", "content": f"{row.context}"}
    answers_list.append(brand_dict)

message_objects.extend(answers_list)
message_objects.append({"role": "assistant", "content":"Here's my answer to your question."})

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)

print(completion.choices[0].message['content'])

The University of Notre Dame offers a wide range of undergraduate and graduate degrees across various disciplines. The College of Arts and Letters offers degrees in areas such as English, history, psychology, sociology, and philosophy, among others. The College of Science offers degrees in fields such as biology, chemistry, physics, mathematics, and pre-professional studies. The College of Engineering offers degrees in aerospace engineering, civil engineering, computer science, electrical engineering, and more. The Mendoza College of Business offers undergraduate degrees in fields such as finance, marketing, management, and accounting, as well as graduate programs like the MBA. The School of Architecture, as the name suggests, offers degrees in architecture. There are also specialized programs such as the Alliance for Catholic Education program, which offers a Master of Education degree. These are just a few examples, and there are many more degree programs available at the University 